In [1]:
from __future__ import print_function
import torch
import numpy as np

In [2]:
x = torch.rand(5, 3)
print(x)

tensor([[0.2468, 0.6396, 0.4974],
        [0.6675, 0.2989, 0.5365],
        [0.7819, 0.1912, 0.6868],
        [0.5178, 0.9918, 0.3175],
        [0.7511, 0.7079, 0.6365]])


In [3]:
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [4]:
x = torch.tensor([5.5, 3])
print(x)
print(x.shape)

tensor([5.5000, 3.0000])
torch.Size([2])


In [5]:
x = x.new_ones(5, 3, dtype=torch.double)      # new_* methods take in sizes
print(x)
print(x.shape)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
torch.Size([5, 3])


In [6]:
x = torch.randn_like(x, dtype=torch.float)
print(x)

tensor([[ 1.7438, -0.3451,  0.3730],
        [ 0.3841,  0.3344, -0.4584],
        [ 1.0262, -0.6919,  0.1275],
        [-0.2891, -0.9584,  0.1160],
        [ 1.1428,  0.6403,  0.9265]])


In [7]:
print(x.size())
print(x.shape)

torch.Size([5, 3])
torch.Size([5, 3])


In [8]:
y = torch.rand(5, 3)
print(x + y)

tensor([[ 2.4748, -0.1813,  0.6233],
        [ 0.6883,  1.2047,  0.2046],
        [ 1.5071,  0.0147,  0.3200],
        [-0.2549, -0.3255,  0.7428],
        [ 1.3479,  0.6768,  1.0706]])


In [9]:
z = torch.add(x,y)
z

tensor([[ 2.4748, -0.1813,  0.6233],
        [ 0.6883,  1.2047,  0.2046],
        [ 1.5071,  0.0147,  0.3200],
        [-0.2549, -0.3255,  0.7428],
        [ 1.3479,  0.6768,  1.0706]])

In place addition

In [10]:
y.add_(x)
print(y)

tensor([[ 2.4748, -0.1813,  0.6233],
        [ 0.6883,  1.2047,  0.2046],
        [ 1.5071,  0.0147,  0.3200],
        [-0.2549, -0.3255,  0.7428],
        [ 1.3479,  0.6768,  1.0706]])


In [11]:
a = y[:,1]
a

tensor([-0.1813,  1.2047,  0.0147, -0.3255,  0.6768])

In [12]:
a = y[1:3,:]
a

tensor([[0.6883, 1.2047, 0.2046],
        [1.5071, 0.0147, 0.3200]])

For a one element tensor you can use .item() to get the value as a python number

### Numpy

In [13]:
a = torch.ones(5)
print(a)
b = a.numpy()
print(b)

tensor([1., 1., 1., 1., 1.])
[1. 1. 1. 1. 1.]


In [14]:
a.add_(1.)
print(a)
print(b)

tensor([2., 2., 2., 2., 2.])
[2. 2. 2. 2. 2.]


In [15]:
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


In [16]:
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))

tensor([[2.7438, 0.6549, 1.3730],
        [1.3841, 1.3344, 0.5416],
        [2.0262, 0.3081, 1.1275],
        [0.7109, 0.0416, 1.1160],
        [2.1428, 1.6403, 1.9265]], device='cuda:0')
tensor([[2.7438, 0.6549, 1.3730],
        [1.3841, 1.3344, 0.5416],
        [2.0262, 0.3081, 1.1275],
        [0.7109, 0.0416, 1.1160],
        [2.1428, 1.6403, 1.9265]], dtype=torch.float64)


In [15]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from tensorboardX import SummaryWriter
import numpy as np

log_path = './runs/gd/'

if log_path:
    print("accessing predefined path")
    writer = SummaryWriter(log_dir=log_path)
else :
    print("using new path set")
    writer = SummaryWriter(log_dir='./runs/gd/')

dtype = torch.FloatTensor
N, D_in, H, D_out = 64, 1000, 100, 10

x = Variable(torch.randn(N, D_in).type(dtype), requires_grad=False)
y = Variable(torch.randn(N, D_out).type(dtype), requires_grad=False)

w1 = Variable(torch.randn(D_in, H).type(dtype), requires_grad=True)
w2 = Variable(torch.randn(H, D_out).type(dtype), requires_grad=True)

learning_rate = 1e-6
errors = []
weights_1_out = []
weights_2_out = []

num_iterations = 5000

for iteration in range(num_iterations):
    y_pred = F.relu(x.mm(w1).clamp(min=0)).mm(w2)
    

    error = (y_pred - y).pow(2).sum()
    error.backward()
    
    writer.add_scalar(tag="Last run",scalar_value= error, global_step = iteration)
    writer.add_histogram("error distribution",error)

    with torch.no_grad():
        w1.data -= learning_rate * w1.grad.data
        w2.data -= learning_rate * w2.grad.data
        
    if iteration % 50 == 0:
        print("Iteration: %d - Error: %.4f" % (iteration, error.item()))
        weights_1_out.append(w1.cpu().detach().numpy())
        weights_1_out.append(w2.cpu().detach().numpy())
        errors.append(error.cpu().detach().numpy())

    w1.grad.data.zero_()
    w2.grad.data.zero_()
    
    if error.item() < 1e-6:
        print("Stopping gradient descent, algorithm converged, MSE loss is smaller than 1E-6")
        break

accessing predefined path
Iteration: 0 - Error: 23744392.0000
Iteration: 50 - Error: 11550.6650
Iteration: 100 - Error: 323.1526
Iteration: 150 - Error: 16.2050
Iteration: 200 - Error: 1.0760
Iteration: 250 - Error: 0.0843
Iteration: 300 - Error: 0.0075
Iteration: 350 - Error: 0.0009
Iteration: 400 - Error: 0.0002
Iteration: 450 - Error: 0.0001
Iteration: 500 - Error: 0.0000
Iteration: 550 - Error: 0.0000
Iteration: 600 - Error: 0.0000
Iteration: 650 - Error: 0.0000
Iteration: 700 - Error: 0.0000
Iteration: 750 - Error: 0.0000
Iteration: 800 - Error: 0.0000
Iteration: 850 - Error: 0.0000
Iteration: 900 - Error: 0.0000
Iteration: 950 - Error: 0.0000
Iteration: 1000 - Error: 0.0000
Iteration: 1050 - Error: 0.0000
Iteration: 1100 - Error: 0.0000
Iteration: 1150 - Error: 0.0000
Iteration: 1200 - Error: 0.0000
Iteration: 1250 - Error: 0.0000
Iteration: 1300 - Error: 0.0000
Iteration: 1350 - Error: 0.0000
Iteration: 1400 - Error: 0.0000
Iteration: 1450 - Error: 0.0000
Iteration: 1500 - Error